In [85]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import sympy as sym
import time

In [78]:
priorsdf = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
paramsdf = pd.DataFrame(columns = ['name','mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['theKBO','obj1','obj2'])

paramsdf['name'] = ['Haumea','Hi\'aka','Namaka']
paramsdf['mass'] = [4.006*10**21,1.79*10*19,1.79*10**18]
paramsdf['sma'] = [np.NaN,49880,25657]
paramsdf['ecc'] = [np.NaN,0.0513,0.249]
paramsdf['aop'] = [np.NaN,154.1,178.9]
paramsdf['inc'] = [np.NaN,126.356,113.013]
paramsdf['lan'] = [np.NaN,206.766,205.016]
paramsdf['mea'] = [np.NaN,152.8,178.5]
paramsdf['j2r2'] = [np.NaN,4,5]
paramsdf['c22r2'] = [np.NaN,4,6]
paramsdf['spaop'] = [np.NaN,4,5]
paramsdf['spinc'] = [np.NaN,2,1]
paramsdf['splan'] = [np.NaN,4,6]

priorsdf['mass'] = [3,0,10,1,2,1,1,5*10**19,10*20]
priorsdf['sma'] = [2,0,12,1,2,35000,20000,1,1]
priorsdf['ecc'] = [0,0,1,0,1,1,1,1,1]
priorsdf['aop'] = [0,0,180,1,2,1,1,1,1]
priorsdf['inc'] = [0,0,180,1,2,1,1,1,1]
priorsdf['lan'] = [0,0,360,0,12,1,1,1,1]
priorsdf['mea'] = [0,0,180,1,2,1,1,1,1]
priorsdf['j2r2'] = [2,0,12,1,2,1,1,1,1]
priorsdf['c22r2'] = [2,0,12,1,8,1,1,1,1]
priorsdf['spaop'] = [2,0,16,1,6,1,1,1,1]
priorsdf['spinc'] = [2,0,12,0,8,1,1,1,1]
priorsdf['splan'] = [2,0,12,1,8,1,1,1,1]

print(paramsdf,priorsdf)
print(paramsdf.transpose(),priorsdf.transpose())

          name          mass      sma     ecc    aop      inc      lan    mea  \
theKBO  Haumea  4.006000e+21      NaN     NaN    NaN      NaN      NaN    NaN   
obj1    Hi'aka  3.401000e+02  49880.0  0.0513  154.1  126.356  206.766  152.8   
obj2    Namaka  1.790000e+18  25657.0  0.2490  178.9  113.013  205.016  178.5   

        j2r2  c22r2  spaop  spinc  splan  
theKBO   NaN    NaN    NaN    NaN    NaN  
obj1     4.0    4.0    4.0    2.0    4.0  
obj2     5.0    6.0    5.0    1.0    6.0                                    mass    sma  ecc  aop  inc  lan  mea  j2r2  \
dist. shape                         3      2    0    0    0    0    0     2   
uni-low                             0      0    0    0    0    0    0     0   
uni-up                             10     12    1  180  180  360  180    12   
log-uni-low                         1      1    0    1    1    0    1     1   
log-uni-up                          2      2    1    2    2   12    2     2   
norm-cen                     

In [80]:
def mm_priors(priors, params):
    columnList = list(priors)
    totalLogProb = 0
    
    probDist = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['PDF'])
   
    paramsList = params.to_numpy().tolist()
    count = 0
    allProbs = []
    numNaNs = 0
    #This loop runs through every column in the priors dataframe, and evaluates the probability density
    #function of the specified type.
    for i in columnList:
        count += 1
        
        #Uniform Distribution Shape
        if priors[i][0] == 0:
            for x in paramsList:
                if x[count] < priors[i][2] and x[count] > priors[i][1]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
        
        #Log-Uniform Distribution Shape
        elif priors[i][0] == 1:
            for x in paramsList:
                if x[count] < priors[i][4] and x[count] > priors[i][3]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
            
        # Normal Distribution Shape
        elif priors[i][0] == 2:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*((x[count]-priors[i][6])/priors[i][5])**2))
            
        #Log Normal Distribution Shape
        elif priors[i][0] == 3:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*(((np.log(x[count])-priors[i][8])**2)/(priors[i][7])**2))/x[count])
        else:
            print('Invalid input.') 
            
        #Here, add the Prior Probability Density function for this element to the total
    for x in allProbs:
        totalLogProb = totalLogProb + np.log(x)
        print(x)
        
    return totalLogProb

In [84]:
dist = mm_priors(priorsdf, paramsdf)
print('Total Log Probability: ', dist)


2.4962556165751374e-22
0.002940311673037342
5.58659217877095e-19
0.69460354097969
0.9870230388205508
1
1
1
1
1
1
1
1
1
1
0.011108996538242306
0.00033546262790251185
0.011108996538242306
3.726653172078671e-06
0.011108996538242306
0.00033546262790251185
0.6065306597126334
1.0
0.011108996538242306
3.726653172078671e-06
Total Log Probability:  -157.47754311424458


C:\Users\annas\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater
